In [1]:
import os
import sys
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F  
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

In [2]:
EPSILON = 0.1       # Maximum perturbation (for L∞ norm)
ALPHA = 0.01        # Learning rate or step size for universal update
ATTACK_ITERATIONS = 40  # PGD inner iterations
TARGET_LABEL = 1    # The target label for all attacks

# Data window parameters
WINDOW_SIZE = 10000
HOP_SIZE = 500
START_INDEX = 4800
END_INDEX = 6000

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
MODEL_PATH = "/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/weights/best_model_retrained.pth"

# Add your model path so that you can import your network
sys.path.append("/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/models")
from attempt2 import resnet50_1d  # Import your model definition

num_classes = 8
model = resnet50_1d(num_classes=num_classes).to(DEVICE)
print(f"Loading model weights from: {MODEL_PATH}")
state_dict = torch.load(MODEL_PATH, map_location=DEVICE)
model.load_state_dict(state_dict)
model.eval()



In [4]:
class IQDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]

        sample = torch.from_numpy(sample).float()
        # Normalize data
        magnitude = torch.sqrt(torch.sum(sample**2, dim=1, keepdim=True))
        sample = sample / magnitude

        label_tensors = torch.tensor(label, dtype=torch.long)

        return sample, label_tensors

In [5]:
def load_iq_files(file_paths, start_index, end_index, hop_size, window_size):
    """
    Load IQ data from a list of files and extract windows using a sliding window.
    The label is extracted from the file name by splitting on '_' (the second token is used).
    """
    data_list = []
    labels_list = []

    for file_path in file_paths:
        print(f"Processing file: {file_path}")
        # Read IQ data
        data = np.fromfile(file_path, dtype="float32")
        real = data[0::2]
        imag = data[1::2]
        # Determine label from file name (e.g., Pluto_0_2m_run2.iq → label 0)
        fname = os.path.basename(file_path)
        try:
            label = int(fname.split("_")[1])
        except (IndexError, ValueError):
            print(f"Warning: Could not extract label from filename {fname}; defaulting to -1.")
            label = -1

        # Create sliding windows
        for x in range(start_index, end_index):
            start = (x + 1) * hop_size
            end_win = start + window_size
            if end_win > len(real):
                break
            i_window = real[start:end_win]
            q_window = imag[start:end_win]
            combined = np.vstack((i_window, q_window))  # Shape: [2, window_size]
            data_list.append(combined)
            labels_list.append(label)

    if not data_list:
        raise ValueError("No valid data was loaded from the provided files.")
        
    data_array = np.stack(data_list, axis=0)
    labels_array = np.array(labels_list)
    return IQDataset(data_array, labels_array)

In [6]:
def generate_targeted_uap(model, data_loader, target_label, eps, alpha, num_epochs, device):
    """
    Generate a targeted universal adversarial perturbation (UAP) using direct gradient updates.
    The universal perturbation is shared across all examples.
    """
    # Initialize universal perturbation delta [1, 2, WINDOW_SIZE]
    universal_delta = torch.zeros(1, 2, WINDOW_SIZE, device=device, requires_grad=True)
    optimizer = torch.optim.SGD([universal_delta], lr=alpha)

    for epoch in range(num_epochs):
        print(f"UAP Epoch {epoch + 1} / {num_epochs}")
        for x, _ in data_loader:
            x = x.to(device)  # x shape: [B, 2, WINDOW_SIZE]
            batch_size = x.size(0)
            # Expand delta over the batch
            delta_batch = universal_delta.expand_as(x)
            x_adv = x + delta_batch

            # Create target labels for all examples
            target = torch.full((batch_size,), target_label, device=device, dtype=torch.long)
            outputs = model(x_adv)
            preds = torch.argmax(outputs, dim=1)
            # Only update on samples not yet fooled (not classified as target)
            mask = (preds != target)
            if mask.sum().item() > 0:
                x_masked = x[mask]
                # Use the same universal delta for these samples (broadcasted)
                x_adv_masked = x_masked + universal_delta.expand_as(x_masked)
                outputs_masked = model(x_adv_masked)
                loss = F.cross_entropy(outputs_masked, target[mask])
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                # Project the universal perturbation into the L∞ ball
                with torch.no_grad():
                    universal_delta.data = torch.clamp(universal_delta.data, -eps, eps)

    return universal_delta.detach()

In [ ]:
if __name__ == "__main__":
    # List the three IQ file paths (update these paths as needed)
    iq_files = [
        "/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/1m_2m_replacedPluto4/1m_2m/Pluto_0_2m_run2.iq",
        "/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/1m_2m_replacedPluto4/1m_2m/Pluto_0_2m_run3.iq",
        "/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/1m_2m_replacedPluto4/1m_2m/Pluto_0_2m_run4.iq"
    ]
    
    # Load the dataset from the given files
    dataset = load_iq_files(
        file_paths=iq_files,
        start_index=START_INDEX,
        end_index=END_INDEX,
        hop_size=HOP_SIZE,
        window_size=WINDOW_SIZE
    )
    BATCH_SIZE = 16
    data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    # --- Option 1: Generate UAP using direct gradient updates ---
    NUM_EPOCHS_UAP = 5  # Increase this for better optimization
    print("\nGenerating targeted UAP (without inner-loop PGD)...")
    uap_noise = generate_targeted_uap(
        model=model,
        data_loader=data_loader,
        target_label=TARGET_LABEL,
        eps=EPSILON,
        alpha=ALPHA,
        num_epochs=NUM_EPOCHS_UAP,
        device=DEVICE
    )
    # Save universal perturbation for inspection/application
    np.save("uap_target1.npy", uap_noise.cpu().numpy())
    print("Saved targeted UAP to uap_target1.npy")
    
    # --- Option 2: Generate UAP using PGD refinement on misclassified samples ---
    NUM_EPOCHS_UAP_PGD = 5  # Increase this if needed for convergence
    print("\nGenerating targeted UAP with PGD refinement...")
    uap_pgd_noise = generate_targeted_uap_with_pgd(
        model=model,
        data_loader=data_loader,
        target_label=TARGET_LABEL,
        eps=EPSILON,
        alpha=ALPHA,
        attack_iter=ATTACK_ITERATIONS,
        num_epochs=NUM_EPOCHS_UAP_PGD,
        device=DEVICE
    )
    # Save the UAP+PGD noise
    np.save("uap_pgd_target1.npy", uap_pgd_noise.cpu().numpy())
    print("Saved targeted UAP with PGD refinement to uap_pgd_target1.npy")

In [ ]:
generate noise pgd
attenaute 
test
update weights


Maybe also have pgd work for all three runs